<a href="https://colab.research.google.com/github/JayThibs/gpt-experiments/blob/main/notebook/gpt_2_alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning GPT-2 on Alignment Texts Dataset

This notebook is meant for initial experimentation of fine-tuning on the alignment text dataset.

In [1]:
!nvidia-smi

Mon Jul  4 17:50:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installations

In [2]:
!pip install git+https://github.com/huggingface/transformers pytorch-lightning beautifulsoup4 datasets jsonlines ftfy lm_dataformat wandb --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 585 kB 4.1 MB/s 
     |████████████████████████████████| 362 kB 80.8 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 1.8 MB 71.6 MB/s 
     |████████████████████████████████| 6.6 MB 83.5 MB/s 
     |████████████████████████████████| 596 kB 91.8 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 140 kB 85.7 MB/s 
     |████████████████████████████████| 419 kB 85.0 MB/s 
     |████████████████████████████████| 1.1 MB 70.6 MB/s 
     |████████████████████████████████| 212 kB 92.8 MB/s 
     |████████████████████████████████| 127 kB 93.4 MB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 2.5 MB 23.3 MB/s 
     |████████████████████████████████| 145 kB 89.3 MB/s 
     |███

# Imports

In [3]:
import os
import re
import torch
import random
import jsonlines
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import Dataset
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2TokenizerFast, AutoTokenizer, TrainingArguments, Trainer, GPT2LMHeadModel
import ftfy
from lm_dataformat import Reader
pd.set_option('display.max_colwidth', None)

# Mounting Google Drive

Here we will mount our Google Drive so that we can grab data and save the HuggingFace scripts, and save the model once we've fine-tuned it.

In [4]:
# For saving the data locally
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd drive/MyDrive/data/ai-alignment-dataset/

/content/drive/MyDrive/data/ai-alignment-dataset


In [6]:
# !git clone https://github.com/JayThibs/gpt-experiments

# Data Preparation

## Preparing Sub-Datasets

In [13]:
with jsonlines.open("alignment_forum.jsonl", "w") as writer:
    with jsonlines.open("alignment_texts.jsonl") as reader:
        for line in reader:
            try:
                if line["source"] == "alignment forum":
                    writer.write(line)
            except:
                pass

## Clearning and Chunking Functions

Functions for preparing the data into chunks that can fit into GPT.

In [14]:
!python create_finetune_csv.py "alignment_texts.jsonl" "gpt" --normalize-with-ftfy --min-unique-tokens=10

Downloading: 100% 0.99M/0.99M [00:00<00:00, 1.15MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 506kB/s]
Downloading: 100% 1.29M/1.29M [00:01<00:00, 1.24MB/s]
Downloading: 100% 665/665 [00:00<00:00, 593kB/s]
reading/tokenizing files: 100% 2138/2138 [00:36<00:00, 58.09it/s]
enforce_min_unique_tokens: 100% 7288/7288 [00:00<00:00, 28910.39it/s]
7288
1000
dropped 920 tokens of trailing data


In [11]:
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [12]:
# import csv

# i = 0
# texts = []
# with jsonlines.open("alignment_texts.jsonl") as reader:
#     for line in reader:
#         text = line["text"]
#         texts.append(text)
#         if i > 3:
#             break
#         # try:
#         if text != "":
#             print(text)
#             print(len(text.split()))
#             encoding = tokenizer(text)
#             total_len = len(encoding.tokens())
#             tokens = encoding.tokens()
#             # print(tokens)
#             print(tokenizer.decode(encoding.input_ids))
#         # if total_len > 1024:
#         #     break
#         i += 1
#         # except:
#         #     pass

## Training Splits

In [17]:
alignment_texts = pd.read_csv("alignment_texts_7288.csv")

In [18]:
alignment_texts = list(alignment_texts)
alignment_texts[0]

'<|endoftext|> I\'ll be running an Ask Me Anything on this post from Friday (April 30) to Saturday (May 1).\nIf you want to ask something just post a top-level comment; I\'ll spend at least a day answering questions.\nYou can find some background about me here.\n<|endoftext|>**I—Meanings**\nNow that we have some more concrete thinking under our belt, it\'s time to circle back on Goodhart\'s law for value learners. What sorts of bad behavior are we imagining from future value-learning AI? What makes those behaviors plausible, and what makes them bad?\nLet\'s start with that last point first. Judgments of goodness or badness get contextualized by models, so our framing of Goodhart\'s law depends on what models of humans we tolerate. When I say "I like dancing," this is a different use of the word \'like,\' backed by a different model of myself, than when I say "I like tasting sugar." The model that comes to mind for dancing treats it as one of the chunks of my day, like "playing computer

In [ ]:
train, val = train_test_split(musk_tweets, test_size=0.2)
test, val = train_test_split(val, test_size=0.5)

In [ ]:
print("Number of Train examples: " + str(len(train)))
print("Number of Val examples: " + str(len(val)))
print("Number of Test examples: " + str(len(test)))

Number of Train examples: 27148
Number of Val examples: 3394
Number of Test examples: 3393


In [ ]:
train_path = f'{directory}' + 'train.csv'
val_path = f'{directory}' + 'val.csv'
test_path = f'{directory}' + 'test.csv'

train.to_csv(train_path, index=False)
val.to_csv(val_path, index=False)
test.to_csv(test_path, index=False)

# Fine-Tuning GPT-2

If we're looking to fine-tune models which are found on the HuggingFace model hub, then it becomes much easier to fine-tune the models since HuggingFace provides us with scripts.

From the `transformers` repo:

> There are two sets of scripts provided. The first set leverages the Trainer API. The second set with no_trainer in the suffix uses a custom training loop and leverages the 🤗 Accelerate library. Both sets use the 🤗 Datasets library. You can easily customize them to your needs if you need extra processing on your datasets.

You can learn more about it here: https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling

We will be using the script that leveraged the Trainer API. We can download the script by running:

In [9]:
if not os.path.exists('/gpt-2/run_clm.py'):
    !wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/language-modeling/run_clm.py -P gpt-2/

--2022-07-02 21:00:52--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25025 (24K) [text/plain]
Saving to: ‘gpt-2/run_clm.py.1’

run_clm.py.1        100%[===================>]  24.44K  --.-KB/s    in 0.003s  

2022-07-02 21:00:52 (7.72 MB/s) - ‘gpt-2/run_clm.py.1’ saved [25025/25025]



# Train

In [19]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [20]:
!python gpt-2/run_clm.py \
    --model_name_or_path "gpt-2/tmp/alignment-texts-clm" \
    --train_file alignment_texts_7288.csv \
    --do_train \
    --fp16=True \
    --overwrite_cache=True \
    --per_device_train_batch_size=2 \
    --output_dir gpt-2/tmp/alignment-forum \
    --overwrite_output_dir="no" \
    --save_total_limit=1 \
    --gradient_accumulation_steps=8 \
    --warmup_steps=10 \
    --learning_rate=3e-5 \
    --weight_decay=0.1 \
    --report_to="wandb" \
    --run_name="gpt-2-alignment-forum-20220703"

07/04/2022 01:24:36 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
07/04/2022 01:24:36 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=8,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_r

In [20]:
# !python gpt-2/run_clm.py \
#     --model_name_or_path gpt2 \
#     --train_file alignment_texts_87606.csv \
#     --do_train \
#     --fp16=True \
#     --overwrite_cache=True \
#     --per_device_train_batch_size=2 \
#     --output_dir gpt-2/tmp/alignment-texts-clm \
#     --overwrite_output_dir="yes" \
#     --save_total_limit=3 \
#     --save_steps=10000 \
#     --gradient_accumulation_steps=32 \
#     --warmup_steps=100 \
#     --learning_rate=3e-5 \
#     --weight_decay=0.1 \
#     --report_to="wandb" \
#     --run_name="gpt-2-alignment-20220702"

07/02/2022 21:08:15 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
07/02/2022 21:08:15 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=32,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_

In [21]:
wandb.finish()

# Let's use the model!

In [11]:
OUTPUT_DIR = "gpt-2/tmp/alignment-forum"
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)

In [16]:
NUM_COMPLETIONS = 1

def generate(input_str, length=50, n=NUM_COMPLETIONS):
  cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
  model.eval()
  with torch.no_grad():
    for i in range(length):
      outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
      loss, logits = outputs[:2]
      softmax_logits = torch.softmax(logits[0,-1], dim=0)
      next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
      cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    return output_text.replace("<|endoftext|>", "")

def choose_from_top(probs, n=NUM_COMPLETIONS):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)


In [17]:
import time

start = time.time()
generated_text = generate("**I—Meanings**\nNow that we have some more concrete thinking under our belt, it\'s time to circle back on Goodhart\'s law for value learners.")
end = time.time()
print(generated_text)
print(end - start)

**I—Meanings**
Now that we have some more concrete thinking under our belt, it's time to circle back on Goodhart's law for value learners.
**Definition 1:** *A learner *(A) *(A) *(A) *(A) *(A) *(A) *(A) *(A) *(A) *(A)
9.844796895980835


# Compressing the Model

Let's save the model as a `tar.gz` file so that we can save it in Google Drive.

In [ ]:
!tar -czf gpt-2-elon-tweets.tar.gz gpt-2/tuned-models/